### Automated Event Planning

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from crewai import Agent, Task, Crew
"""
from crewai import LLM
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )



from langchain_community.llms import Ollama
ollama_llm =Ollama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)
"""
from langchain_openai import ChatOpenAI
import os
ollama_llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,

)

In [19]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

#### Creating Agents

In [20]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",

    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [21]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [22]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=ollama_llm,
    allow_delegation=False
)

### Creating Venue Pydantic Object
Agents will populate this object with information about different venues by creating different instances of it.

In [23]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [24]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator,
   
)

In [25]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    #async_execution=True,
    agent=logistics_manager,
  
)

In [26]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,

)

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [27]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
           
    
    verbose=True
)

2025-05-18 11:07:25,631 - 8417615616 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


#### Define Input


In [30]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2025-05-12",
    'expected_participants': 200,
    'budget': 2000000,
    'venue_type': "Conference Hall"
}

In [ ]:

result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering information about the event and the criteria for the venue.

Action: Search the internet

Action Input: {"search_query": "tech innovation conference san francisco venue criteria"}
 


Search results: Title: Venue - World Agri-Tech Innovation Summit, San Francisco
Link: https://worldagritechusa.com/venue/
Snippet: SUMMIT VENUE: MARRIOTT MARQUIS. World Agri-Tech is returning to California, the global epicenter for innovation discovery and investment.
---
Title: TECHSPO San Francisco 2025
Link: https://techsposanfrancisco.com/
Snippet: Join us on July 21 – 22, 2025, at the Grand Hyatt Hotel at SFO in San Francisco for 2 unforgettable days of technology exploration.
---
Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechu

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '', 'booking_status': '', 'capacity': 0, 'name': 'Human'}
